In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
from tqdm import tqdm
import string
import pandas as pd
import random
import numpy as np

In [3]:
# MUST DOWNLOAD EdgeDriver, ChromeDriver, or some other executable
# store path to executable below
DRIVER_PATH = 'msedgedriver.exe'
driver = webdriver.Edge(DRIVER_PATH)

C:\Users\anany\AppData\Local\Temp\ipykernel_7860\3676954888.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Edge(DRIVER_PATH)


In [5]:
df = pd.read_csv('spotify-2023.csv', encoding='ISO-8859-1')

In [ ]:
lyricslist = []
base_url = "https://www.azlyrics.com/lyrics/{}/{}.html"

for row in tqdm(df[['track_name', 'artist(s)_name']].values):
    artists = row[1].split(',')
    lyrics = None
    title = row[0]

    for artist in artists:
        artist = artist.lower()

        if artist[:4] == "the ":
            artist = ''.join(artist.split('the '))

        artist = ''.join(artist.split(' '))
        artist = artist.translate(str.maketrans('', '', string.punctuation))

        title = re.sub(r'[\(\[].*?[\)\]]', '', row[0])
        title = ''.join(title.split(' '))
        title = title.translate(str.maketrans('', '', string.punctuation)).lower()

        if ' - ' in title:
            title, _ = title.split(' - ')

        if '-' in title:
            title = ''.join(title.split('-'))

        url = base_url.format(artist, title)

        try:
            driver.get(url)
            driver.implicitly_wait(random.randint(8, 13))
            lyrics = driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/div[2]/div[5]').text

        except Exception as e:
            pass

        finally:
            if lyrics is not None:
                break

    lyricslist.append(lyrics)

In [ ]:
df['Lyrics'] = lyricslist
df = df.fillna(value=np.nan)
df.to_csv("spotify_with_lyrics.csv", encoding='UTF-8', index=False)